# Final Assignment

# Correlation Between Companies in The Same Industry

Summary of Contents

- [1 Introduction](#toc_1)
- [2 Questions 2 heading](#toc_2)
- [3 Import Required Libraries](#toc_3)


<a name="toc_1"></a>
# Introduction

intro intro intro

# Data Sources

This project uses the following data sets the information contained within are publicly available:

1. sp500_info.csv: 
    * S&P 500 component stocks including their symbol and sector name, and sub sector name <br>
    (Wikipedia 2023, https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)  
    
2. asx_15_info.csv: 
    * Top 20 Shares by Value for the ASX <br>
    (ASX 2023, https://www.asx.com.au/data/dw_sharesbyvalue.pdf)  
      
3. prices_shares.csv: <br>
    * 12 Months Historical Data for 100 S&P500 shares and 15 ASX shares (in their local currency): <br>
    (Aroussi, 2023; Yahoo Finance, 2023)  
        
4. prices_fx.csv: 
    * 12 Months Historical Data for a basket of currencies: <br>
    (Aroussi, 2023; Yahoo Finance, 2023)
    

# Style Testing

<p style="background-color:Yellow;">test</p>

<p style="background-color:#B8E2F2;">
    <b>Finding:</b> test test test 
</p>

<a name="toc_2"></a>
# Questions

What is the break down by industry of the sample set of companies taken from the S&P500.

Given a sample of 115 diferent stocks:
* How many different pairs are possible to analyze
* How many different pairs within the same industry are possible to analyze and compare to the 'total' universe of stocks
 
What is the overall correlation profile of the pairs within the sample?  

Are pairs in the same industry more closely correlated in their price movements than random pairs?

Do pairs in some industries show better correlation than others?

If there is a difference in correlation across industries, are there any apparent reasons for this or additional steps that could be taken to try and determine those?

<a name="toc_3"></a>
# Import Required Libraries

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

In [2]:
import itertools

In [3]:
from random import sample

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

# Set Inputs

In [ ]:
# Target Input Files
STOCK_PRICES_FNAME = "prices_shares.csv"
FX_PRICES_FNAME = "prices_fx.csv"
SP500_MEMBERS_FNAME = "sp500_info.csv"
ASX_MEMBERS_FNAME = "asx_15_info.csv"

#FILEPATH = "C:\\Dropbox\\Variance\\UNSW\\ZZEN9021\\"

In [ ]:
"""
cols = ['Symbol', 'Security','GICS Sector']

data = [
['BHP.AX', 'BHP GROUP' ,'Materials'],
['FMG.AX', 'FORTESCUE', 'Materials'],
['MQG.AX', 'MACQUARIE GROUP', 'Financials'],
['CBA.AX', 'COMMONWEATH BANK', 'Financials'],
['RIO.AX', 'RIO TINTO', 'Materials'],
['CSL.AX', 'CSL', 'Health Care'],
['WDS.AX', 'WOODSIDE','Energy'],
['ANZ.AX', 'ANZ GROUP', 'Financials'],
['PLS.AX', 'PILBARAMIN', 'Materials'],
['NAB.AX', 'NATIONAL AUSTRALIA BANK', 'Financials'],
['WBC.AX', 'WESTPAC ', 'Financials'],
['RMD.AX', 'RESMED', 'Health Care'],
['NCM.AX', 'NEWCREST', 'Materials'],
['NST.AX', 'NTH STAR', 'Materials'],
['GMG.AX', 'GOODMAN GROUP', 'Real Estate'],
]

df = pd.DataFrame(data = data, columns=cols)

df.to_csv('asx_15_info.csv', index=False)
df
"""


In [ ]:
# List of tickers/symbols of currencies that we wish to handle
FX_TICKERS = ["AUDUSD=X", "HKDUSD=X", "JPYUSD=X", "SGDUSD=X"]

The key data we are accessing is pricing information for stocks and fx rates.

Periodically we need to refresh this information as components of the indexes change, corporate actions occur etc.

However, obtaining this information requires downloading it and thus we want to limit that to when required and not download it every time we run the program.

We thus set a flag (REFRESH_DATA) which we can adjust before running the program identifying  whether to refresh our data or not.

In [ ]:
# When running within Ed this needs to remain as False as their are restrictions on accessing external web sites
REFRESH_DATA = True

Set constants to determine limits on file size and number of pairs processed to limit processing and memory requirements.

In [ ]:
# Instead of choosing all 500 names from the S&P500, pick a selection to reduce the file sizes
SP500_SAMPLE_SIZE = 100

# From our list of all generated pairs, select a random sample to reduce processing requiremens
#RDM_SAMPLE_SIZE = 20000
RDM_SAMPLE_SIZE = 4000

# Obtain Names in SP500

If the refresh data flag is set to True, we run the function to scrape the list of names in the S&P500 from Wikipedia and save this to a csv file.

If flag is set to false, the program will use the current version of the csv file.

In [ ]:
def obtain_sp500_info():
    """
    Function to Scrape Details About S&P500 Components from Wikipedia.
    Saves the current S&P Components into a csv file.
    """

    # Scrape the list of components companies in the S&P500 as per Wikipedia
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    
    # Take the first item in the list obtained from table which contains a dataframe of current S&P components 
    df = table[0]
      
    # Save the component stocks to a csv file
    df.to_csv(SP500_MEMBERS_FNAME, index=False)
    
# If refresh flag is True, download current data via running the obtain_sp_500_info function
if REFRESH_DATA:
    obtain_sp500_info()
    

Read the Most current version of the csv file that contains the S&P component information.

In [ ]:
sp500_info_df = pd.read_csv(SP500_MEMBERS_FNAME)
sp500_info_df

In [ ]:
# Trim the number of S&P500 members we are going to review given processing constraints

sp500_info_df = sp500_info_df[:SP500_SAMPLE_SIZE].copy()

Review the number of members to check that it equals the input constant SP500_SAMPLE_SIZE and also that there are no duplicate symbols

In [ ]:
print(SP500_SAMPLE_SIZE)
print(sp500_info_df['Symbol'].count())
print(len(sp500_info_df['Symbol'].unique()))

<span style="color:green">&#x2705;</span>

Check for any periods in the stock names for US listed stocks (which includes all our S&P500 members).  Yahoo Finance, which is where we download the prices from, uses the convention that A shares and B shares utilize dashes in their names instead of periods.

In [ ]:
sp500_info_df[sp500_info_df['Symbol'].str.contains(".", regex=False)]

In [ ]:
# We need to replace periods with dashs within the Symbol columns for S&P500 related names as per Yahoo Finance convention
sp500_info_df['Symbol'] = sp500_info_df['Symbol'].str.replace(".", "-", regex=False)

# Check to ensure this was successful
sp500_info_df[sp500_info_df['Symbol'].str.contains(".", regex=False)]

<span style="color:green">&#x2705;</span>

# Obtain Names in ASX

In [ ]:
asx_info_df = pd.read_csv(ASX_MEMBERS_FNAME)
asx_info_df

# Merge ASX and S&P500 Names 

In [ ]:
all_stocks_info_df = pd.concat([asx_info_df, sp500_info_df[['Symbol', 'Security', 'GICS Sector']]])
all_stocks_info_df                    

In [ ]:
all_stocks_info_df['GICS Sector'].unique()

# Obtain Prices From CSV or Yahoo Finance

If we have chosen to refresh the data, we will now download the latest prices from Yahoo Finance using the yfinance Library.

We will download prices for both our stocks and also for the Foreign Exchange (FX) rates.

We will then update the pricing related csv files.

In [ ]:
if REFRESH_DATA:
    
    # Download stock data and FX data from yfinance
    stock_prices_df = yf.download(list(all_stocks_info_df['Symbol']), start="2022-01-01", end="2022-12-31")["Adj Close"]
    raw_fx_prices_df = yf.download(FX_TICKERS, start="2022-01-01", end="2022-12-31")["Adj Close"]
    
    # Write new prices to csv
    stock_prices_df.to_csv(STOCK_PRICES_FNAME)
    raw_fx_prices_df.to_csv(FX_PRICES_FNAME)



Whether we have just refreshed the prices or not, we will read the share and FX prices from their csv files.  If refreshed, they will contain the data just downloaded.  If not, the prior version of the file.

In [ ]:
# Read the Daily Prices for the Stocks and FX Rates from their respective csv files
stock_prices_df = pd.read_csv(STOCK_PRICES_FNAME)
raw_fx_prices_df = pd.read_csv(FX_PRICES_FNAME)
    

## Review the Raw FX Data

In [ ]:
raw_fx_prices_df.describe()

It quite difficult to analyze the JPY and HKD columns given they are inverted vs how they are conventionally displayed.

Inverting them for reasonableness analysis:

In [ ]:
fx_summ = raw_fx_prices_df.describe()
# Invert FX Rate for display purposes for JPY and HKD to what is conventionally displayed
fx_summ.loc[['mean', 'min', 'max'],['JPYUSD=X', 'HKDUSD=X']] \
            = 1 / fx_summ.loc[['mean', 'min', 'max'],['JPYUSD=X', 'HKDUSD=X']]
fx_summ


<div class="alert alert-block alert-success">
Min and max ranges as well as distributions seem in line with the known characteristics of the currencies
</div>

<div class="alert alert-block alert-danger">
We do not have the same number of records for each currency
</div>

<p style="background-color:Yellow;">test</p>

&#10060;

To fix this we will resample the data using the ffil() function.
First though we'll create a copy of the data to avoid changing original data removing need to reload if rerunning workbook

In [ ]:
 fx_prices = raw_fx_prices_df.copy()

Prior to doing the resampling and for later analysis it is better if we convert dates from being string values a column,  to Datetime values and also set them as the index.

In [ ]:
# Format the Date and set as the Index
fx_prices['Date'] = pd.to_datetime(fx_prices['Date'], format = "%Y-%m-%d")
fx_prices = fx_prices.set_index('Date', drop=True)

In [ ]:
# Resampling data to fill any missing values
# This will also give a price for the week-ends, but that will get dropped later when we merge with the share prices df
fx_prices = fx_prices.resample('D').last().ffill()

In [ ]:
# Check the column counts to ensure they are now consistent
fx_prices.describe().loc['count']

&#x2705;

Here is how our FX data now looks:

In [ ]:
fx_prices

&#x2705;

## Review the Stock Price Data

In [ ]:
stock_prices_df

**Lets Review the price values for reasonableness**

In [ ]:
print(f"Minimium Share Price Value Across All Stocks: ${stock_prices_df.select_dtypes(include='number').min().min():,.02f}")
print(f"Maximum Share Price Value Across All Stocks: ${stock_prices_df.select_dtypes(include='number').max().max():,.02f}")
print(f"Mean Share Price Value Across All Stocks: ${stock_prices_df.select_dtypes(include='number').mean().mean():,.02f}")

The range of values for individual shares prices is quite large and thus direct comparision of absolute values would required scaling of the prices.
However, the range seems reasonable given the dataset - there are no negative values and no extremely large or small values.

&#x2705;

Lets create a scatter plot to visualize the price data

In [ ]:
av_prices_by_stock = stock_prices_df.select_dtypes(include='number').mean()
av_prices_by_stock.index = pd.Series(range(len(av_prices_by_stock)))
plt.scatter(x = av_prices_by_stock.values,
            y = av_prices_by_stock.index,
           )
plt.xlabel('Price')
plt.ylabel('Company Number')
plt.title('Mean Price for Each Company')

<div class="alert alert-block alert-warning">
<b>Not Great:</b> That does not look so great with a small number of obvious outliers.
</div>

Lets check to see if these represent errors or just outliers.

In [ ]:
# Largest two price values by stock
stock_prices_df.select_dtypes(include='number').max().sort_values()[-2:]


Checking these manually to price sources **, the prices look accurate.  Thus the values represent outliers rather than errors.

While there are several ways to handle these outliers depending the analysis being performed and the reason for the outliers, for our analysis we are going to retain them in our data set.

** 
https://finance.yahoo.com/quote/NVR?p=NVR&.tsrc=fin-srch
https://finance.yahoo.com/quote/BKNG?p=BKNG&.tsrc=fin-srch

**Examing the Number of Records for Each Stock**

In [ ]:
print(stock_prices_df.describe().loc['count'].describe())

<div class="alert alert-block alert-success">
The maximum number of rows is 252, and this is the case for >75% of the rows.  As a reasonableness check, there are 260 (52 * 5) week days in the year and some of these will be public holidays.  So we expect somewhere around 250 trading days in a year given stock markets are not open on week-ends and we have to then deduct for public holidays - so 252 looks reasonable.
</div>

# Format Share Pricing Data

## Convert Share Prices to USD Equivalent

In [ ]:
#Then we convert the dates to datetime format and set them as the index

stock_prices_df['Date'] = pd.to_datetime(stock_prices_df['Date'], format = "%Y-%m-%d")
stock_prices_df = stock_prices_df.set_index('Date', drop=True)

Now we add a column for the FX rate applicable given the ticker code format:
* Australian stocks have .AX in their ticker, 
* Japanese stocks have .T, 
* Hong Kong stocks .HK, 
* Singapore stocks .SI; and
* US stocks have no period in their ticker

In [ ]:
# Add currency column based on ticker code format
for ticker in stock_prices_df.columns:
    if ".AX" in ticker:
        stock_prices_df[ticker] = stock_prices_df[ticker] * fx_prices["AUDUSD=X"]
    elif ".HK" in ticker:
        stock_prices_df[ticker] = stock_prices_df[ticker] * fx_prices["HKDUSD=X"]
    elif ".T" in ticker:
        stock_prices_df[ticker] = stock_prices_df[ticker] * fx_prices["JPYUSD=X"]
    elif ".SI" in ticker:
        stock_prices_df[ticker] = stock_prices_df[ticker] * fx_prices["SGDUSD=X"]

## Eliminate shares with less than 245 trading days

In [ ]:
# Identify Names Which Should be Eliminated
low_counts = stock_prices_df.count()[stock_prices_df.count()<250]
print(low_counts)

low_counts = list(low_counts.index)
print(low_counts)

In [ ]:
# Eliminate Names with a Low Number of Trading Days Present
stock_prices_df = stock_prices_df.drop(columns=low_counts).copy()

In [ ]:
# Check removal was successful
stock_prices_df.count().min()

&#x2705;

## Fill Prices for Names Missing only a Small Number of Days

In [ ]:
# Forward fill any missing stock prices
stock_prices_df.fillna(method='ffill', inplace=True)

# Backward fill any remaining missing stock prices
stock_prices_df.fillna(method='bfill', inplace=True)

stock_prices_df

# Create a Universe of Pairs

Now that we have imported and cleansed the data, we need to start manipulating it to get the answers required.

Key to this is breaking the price data for all the companies into pairs that can be analyzed.  For Example, Microsoft (MSFT) vs Google (GOOGL).


Using all the names we have data for, we can create a list of all possible pair combinations using the itertools library.

In [ ]:
all_possible_pairs = [list(pair) for pair in itertools.combinations(stock_prices_df.columns, 2)]

print(f"The Number of All Possible Unique Pairs is: {len(all_possible_pairs):,.0f}")
print(f"\nSome Examples: {all_possible_pairs[:5]} \n{all_possible_pairs[-5:]}")


We may want to instead look at a sample of names, instead of the full universe.  This may be because we have processing constrainsts etc.  However, we want a random sample rather than just picking the first 100 names in case their is some bias in the way they are ordered.

We have set the sample size via the input constant (RDM_SAMPLE_SIZE) in the Set Inputs section.

In [ ]:
# Select a sample of pairs randomly from the our universe of all possible pairs
random_pairs = sample(all_possible_pairs, RDM_SAMPLE_SIZE)
print(random_pairs[:5])

## Add Industry to Pairs List

The key factor we are seeking to test is Industry.  That is, if in names in the same industry show better correlation or related statistics vs random pairs. 

To do this we are going to have to merge information from multiple inputs:
- the ticker names for the pairs will come from the list of pairs we generated;
- the Industry name will come from the all_stocks_info_df
- Same Industry Column will be a engineered column we create

The Same Industry column will make queries and analysis on the dataset simplier.


In [ ]:
# Build a DataFrame which lists all possible pairs, the industry of each name in the pair and if they are the same.

# Start with the all the pair combinations
all_possible_pairs_df = pd.DataFrame(all_possible_pairs, columns=['Ticker1', 'Ticker2'])

# write to csv to check file size
all_possible_pairs_df.to_csv("all_possible_pairs_df.csv")

# Add the Industry Name for the first ticker
all_possible_pairs_df = pd.merge(all_possible_pairs_df, all_stocks_info_df[['Symbol', 'GICS Sector']], how='inner', left_on = 'Ticker1', right_on = 'Symbol')
# Add the Industry name for the second ticker
all_possible_pairs_df = pd.merge(all_possible_pairs_df, all_stocks_info_df[['Symbol', 'GICS Sector']], how='inner', left_on = 'Ticker2', right_on = 'Symbol')

# Remove excess columns and tidy up column names
all_possible_pairs_df.drop(columns = ['Symbol_x', 'Symbol_y'], inplace=True)
all_possible_pairs_df.rename(columns={"GICS Sector_x": "Ind1", "GICS Sector_y": "Ind2"}, inplace=True)

# Create a new column 'Same Industry' which identifies if all the two names in the pair are from the same industry
all_possible_pairs_df['Same Industry'] = np.where(all_possible_pairs_df['Ind1'] == all_possible_pairs_df['Ind2'], 'Yes', 'No')
all_possible_pairs_df

Construct a DataFrame for the Random Pairs sample in the same format as that for all the possible pairs.

In [ ]:
#Build the base df from the list of list of random pairs
random_pairs_df = pd.DataFrame(random_pairs, columns=['Ticker1', 'Ticker2'])

# Add the extra columns required via merging with the df containing all the pairs
random_pairs_df = pd.merge(random_pairs_df, all_possible_pairs_df, how='inner',
                          left_on= ['Ticker1', 'Ticker2'],
                          right_on= ['Ticker1', 'Ticker2'])

random_pairs_df


### Analysis of Names by Sector ###

In [ ]:
all_stocks_info_df['GICS Sector'].describe()

In [ ]:
print(f"The Minimum Number of Members within a Sector Group is: {all_stocks_info_df.groupby('GICS Sector').size().min()}")
print(f"The Median Number of Members within a Sector Group is: {all_stocks_info_df.groupby('GICS Sector').size().median()}")


In [ ]:
all_stocks_info_df.groupby('GICS Sector').size()

### Build and Display Graphs Analysing the Distribution of the Sector Groups

In [ ]:
# Get Counts of The Number of Names Within Each Sector
sector_counts = all_stocks_info_df.groupby('GICS Sector')['Symbol'].count().sort_values(axis=0)

# Bucket The Industries into Those That Have a High Number of Members and "Others"
sector_counts_series = sector_counts[-8:]
sector_counts_series['Other'] = sector_counts_series[:-8].sum()

In [ ]:
# Create a seaborn color palette for our graphs
colors = sns.color_palette("Blues", len(sector_counts_series))

In [ ]:
# Function to Create a Series of Graphs to Show the Distribution and Makeup of the Sector Membership
def display_sector_graphs(colors, sector_counts, sector_counts_series):
    
    fig = plt.figure(figsize=(10, 25))
    fig.suptitle('Sector Category Splits', y = 0.95, fontweight='bold')

    """
    # Display Histogram Showing What Are the Most Common Number of Members Within a Sector Group
    ax = fig.add_subplot(4, 1, 1)
    ax.hist(sub_industry_counts,
            bins = len(np.arange(1,sub_industry_counts.max())+1),
            )
    ax.set_ylabel('Frequency of Occurence')
    ax.set_xlabel('Number of Members Within Sub-Index')
    ax.set_title('Breakout of the Number of Members Within Each Sector')
    """

    # Display Pie Chart
    ax = fig.add_subplot(3, 1, 1)
    ax.pie(sector_counts_series,
        labels = sector_counts_series.index,
        colors = colors,
            )
    ax.set_title('Sector Split', fontweight='bold')

    # Bar Chart of the Sub-Industries With the Most Members
    ax = fig.add_subplot(3, 1, 2)
    ax.barh(sector_counts_series[:-1].index , sector_counts_series[:-1].values)
    ax.set_title('Top Sector Member Numbers', fontweight='bold')

    # Display Box Plot
    ax = fig.add_subplot(3, 1, 3)
    ax.boxplot(all_stocks_info_df.groupby('GICS Sector')['Symbol'].count())
    ax.set_title('Distribution of Number of Members Within Each Sector', fontweight='bold')

    #fig.tight_layout()
    plt.subplots_adjust(hspace = 0.5)
    
    return fig

When we review the how many stocks/ companies fall into each Sector we see that most Sub-Industries only have a very small number of members.  However, there are also a small number of Sub-Industries that have 10 plus members.

So the distribution is definitely quite skewed.

The following graphs demonstrate that distribution and also highlights which Sector groups have the most number of members.

In [ ]:
fig = display_sector_graphs(colors, sector_counts, sector_counts_series)
#plt.show()

Where the Sector only has a single member, comparing members within the same industry will not be possible.

Given we are looking to compare companies within the same industry vs companies not in the same industry, we want to identify if any sectors have less than 2 names in them.

In [ ]:
all_stocks_info_df.groupby('GICS Sector')['Symbol'].count()

## Analyse Ratio of Same Industry Vs Different Industry Pairs

### Full Universe of Names

In [ ]:
num_pairs_all = len(all_possible_pairs_df.index)
num_pairs_same_ind_all = len(all_possible_pairs_df[all_possible_pairs_df['Same Industry'] == 'Yes'].index)

print(f"Number of Pairs In Total: {num_pairs_all:,}")
print(f"Number of Pairs in the Same Industry: {num_pairs_same_ind_all:,}")
print(f"Percentage of pairs in the same industry: {num_pairs_same_ind_all/num_pairs_all * 100:,.02f}%")

### Sample

In [ ]:
num_pairs_all_samp = len(random_pairs_df.index)
num_pairs_same_ind_samp = len(random_pairs_df[random_pairs_df['Same Industry'] == 'Yes'].index)

print(f"Number of Pairs In Total: {num_pairs_all_samp:,}")
print(f"Number of Pairs in the Same Industry: {num_pairs_same_ind_samp:,}")
print(f"Percentage of pairs in the same industry: {num_pairs_same_ind_samp/num_pairs_all_samp * 100:,.02f}%")

In [ ]:
df = pd.DataFrame(data = [[num_pairs_all, num_pairs_same_ind_all, num_pairs_same_ind_all/num_pairs_all * 100], 
                          [num_pairs_all_samp, num_pairs_same_ind_samp, num_pairs_same_ind_samp/num_pairs_all_samp * 100]], 
                  columns=['Number of Total Pairs', 'Number of Pairs in Same Industry', '% of Pairs in the Same Industry'])
df.index = ['Entire Population', 'Sample']
df

<div class="alert alert-block alert-success">
Our sample does seem representative of the total population in term pairs in the same industry vs all pairs.
</div>

# Correlation

# Calculate The Correlation Between Various Pairs

In [ ]:
corr_array = []
for i in range(len(random_pairs_df)):
    
    #ratio = stock_prices_df[pair[0]]/ stock_prices_df[pair[1]]
    corr = stock_prices_df[random_pairs[i][0]].corr(stock_prices_df[random_pairs[i][1]])
    
    corr_array.append([random_pairs[i][0], random_pairs[i][1], corr])
    
    i += 1
    
corr_df = pd.DataFrame(data=corr_array, columns=['Ticker1', 'Ticker2', 'Correlation'])
corr_df = pd.merge(corr_df, random_pairs_df, how='inner',
                  left_on = ['Ticker1', 'Ticker2'], right_on = ['Ticker1', 'Ticker2'])

corr_df

In [ ]:
stock_prices_df.iloc[:,:5].head()

## Analyze Correlations

We are analyzing the absolute values of the correlations as we are interested in the magnitude of the corrleations - negative correlations are as usual as positive ones.

In [ ]:
corr_all = abs(corr_df['Correlation']).describe()
corr_same_ind = abs(corr_df[corr_df['Same Industry'] == 'Yes']['Correlation']).describe()

df = pd.DataFrame(corr_all)
df = pd.DataFrame({'Corr all': corr_all, 'Corr Same Industry': corr_same_ind})

df.loc['Median', :] \
= [abs(corr_df['Correlation']).median(), abs(corr_df[corr_df['Same Industry'] == 'Yes']['Correlation']).median()]
df

<div class="alert alert-block alert-info">
<b>Finding:</b> At a summary level there does appear to be a higher correlation level for pairs in the same industry</div>

<p style="background-color:#B8E2F2;">
    <b>Finding:</b> test test test 
</p>

## Analyse by Industry : Are Some Industry Groups Better Than Others?

Build a DataFrame which gives us some summary info by Industry for Pairs Which are Both in the Same Industry

In [ ]:
# Take the df of all correlations, extract same industry pairs and get the median correlation and count for those those pairs
df = corr_df[corr_df['Same Industry'] == 'Yes'].groupby('Ind1').agg(Median =('Correlation', 'median'),\
                                                                   Count=('Correlation','count'))
df = df.reset_index()
df['Median'] = abs(df['Median'])
df = df.sort_values(['Median', 'Count']).set_index('Ind1', drop=True)
df.index.name = ""


If we only have one pair in an industry, it is difficult to make any comparision of that industry vs another.  Thus we will limit our comparison to industries with at least 8 names in it.

In [ ]:
# Limit our target dataframe to only include sectors with 10 or more members
df = df.sort_values('Count')
df = df[df['Count']>=10]

df = df.sort_values('Median')
df

In [ ]:
# Calculate median correlations
medians_same_ind = abs(df['Median'])
medians_all_ind = abs(corr_df.groupby('Ind1')['Correlation'].median())

# We only want to plot the industries that have sufficient same industry pairs
medians_all_ind = medians_all_ind[medians_all_ind.index.isin(df.index)]

# Create figure and axis
fig, ax = plt.subplots(figsize=[10,10])

# Set a width for the bars
bar_width = 0.3

# We have to define bar positions if we want the different plots to be side by side
bar1_positions = np.arange(len(medians_same_ind))
bar2_positions = bar1_positions + bar_width # Puts the position beside the prior bar

# Create our bar charts
ax.barh(bar1_positions, medians_same_ind, height=bar_width, label='Median Correlation by Industry for Pairs in the Same Industry')
ax.barh(bar2_positions, medians_all_ind, height=bar_width, label='Median Correlation by Industry for ALL Pairs')

# Define y labels and title
ax.set_yticks(bar1_positions + bar_width / 2)
ax.set_yticklabels(medians_same_ind.index)
ax.set_xlabel('Median Correlation')
ax.set_ylabel('Industry')
ax.set_title('Median Correlation by Industry')

# Show legend
ax.legend()

# Show plot
#plt.show()

<div class="alert alert-block alert-info">
<b>Finding:</b> The range of correlations does vary signficantly across industries
</div>

<div class="alert alert-block alert-info">
<b>Finding:</b> This graph also reinforces the earlier finding that the correlation levels are higher for pairs which are both in the same industry vs disparate pairs - as the same industry pairs have higher correlation for almost all industries</div>

<div class="alert alert-block alert-info">
<b>Finding:</b> It appears that there seems to be INVERSE correlation between the number of members in the Sector and the Median correlation of Sector Members.
    
One possible explanation for this may be that Sectors with larger numbers of members may also be those defined broadly.  If the Sector is quite broad the business models within that sector are likley to become less closely related.

This would require further data analysis and possibly include trying to create sub sectors to see if that creates higher correlation levels.
</div>

In [ ]:
df.corr()

# References

Each list item requires specific information. 

See https://www.student.unsw.edu.au/node/131 
and https://www.student.unsw.edu.au/node/132.

* List each item in alphabetical order, by author surname.
* Titles should be in italics.
* Each item should have a hanging indent.

Aroussi, R., 2023, yfinance.[python library]  
> Available at https://pypi.org/project/yfinance/(Accessed: 3 August, 2023)

ASX, 2023.  *Top 20 Shares by Value.* [pdf] 
> Available at https://www.asx.com.au/data/dw_sharesbyvalue.pdf (Accessed, August 3 2023).

Wikipedia, 2023. *S&P 500 component stocks.*
> Available at https://en.wikipedia.org/wiki/List_of_S%26P_500_companies (Accessed: July 31 2023)

Yahoo Finance, 2023.  *Stock Market and Foreign Currency Historical Market Data.*  
> Available at:
    https://finance.yahoo.com/quote/[Symbol_Code]/history?p=[Symbol_Code](Accessed: 3 August, 2023)

Yahoo Finance, 2023.  *Company Profile Data Incuding Sector Name.*  
> Available at:
    https://finance.yahoo.com/quote/[Symbol_Code]/profile?p=[Symbol_Code](Accessed: 3 August, 2023)

> Note: 115 different stocks and four different foreign currencies were analyzed and thus the individual symbol codes for each are not listed.


# Discarded Graphs etc

In [ ]:
graph_df = stock_prices_df.iloc[:,:3].copy()

pd.plotting.scatter_matrix(graph_df)

In [ ]:
# A Pairplot on Correlations Might Show Something About Industries

# Not really!

In [ ]:

sns.set()
sns.pairplot(
    #data = corr_df.head(),
    data = corr_df,
    hue = 'Ind1',
    height = 5
        )


In [ ]:
sns.pairplot(
    #data = corr_df.head(),
    data = corr_df,
    hue = 'Same Industry',
    height = 5
    
        )

In [ ]:
# Observe when notebook was last run all the way through successfully

print(datetime.now())